In [ ]:
%%capture
!pip install selenium
!pip install chromedriver-autoinstaller
!pip install jupyternotify

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from chromedriver_autoinstaller import install as chromedriver
%load_ext jupyternotify

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')

driver = webdriver.Chrome(executable_path=chromedriver(), options=options)
driver.get('https://hcs.eduro.go.kr/')

wait = WebDriverWait(driver, 10)

In [ ]:
try:
    file = open('../eduro_userdata', 'r')
except FileNotFoundError:
    temp_driver = webdriver.Chrome(executable_path=chromedriver())
    temp_driver.get('https://hcs.eduro.go.kr/')
    while True:
        users = temp_driver.execute_script('''return localStorage.getItem('users');''')
        cookies = temp_driver.execute_script('''return document.cookie;''')
        password = temp_driver.execute_script('''return document.querySelector('input[type="password"]')?.value;''')
        if users:
            file = open('../eduro_userdata', 'w')
            file.write('\n'.join([users, cookies, password]))
            file.close()
            break
    temp_driver.quit()
else:
    users, cookies, password = file.read().split('\n')
    file.close()
    
driver.execute_script(f'''localStorage.setItem('users', '{users}');''')
driver.execute_script(f'''document.cookie = '{cookies}';''')
driver.refresh()

In [ ]:
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[type="password"]'))).send_keys(password)
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#btnConfirm'))).click()

wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.btn'))).click()

wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id$="a1"]')))
answers = driver.find_elements(By.CSS_SELECTOR, 'input[id$="a1"]')
for answer in answers:
    answer.click()
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="submit"]'))).click()

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'img[alt="정상"]')))

%notify -m "자가진단을 완료했습니다."

In [ ]:
users = driver.execute_script('''return localStorage.getItem('users');''')
cookies = driver.execute_script('''return document.cookie;''')
if users:
    file = open('../eduro_userdata', 'w')
    file.write('\n'.join([users, cookies, password]))
    file.close()

In [ ]:
driver.quit()